In [1]:
import numpy as np
import pandas as pd
import re
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
movies_df = pd.read_csv('movies data.csv')
movies_df = movies_df.head(20000) #limiting due to memory issues

In [45]:
movies_df['index'] = range(1,20001)
movies_df.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,...,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations,index
0,760161,Orphan: First Kill,Horror-Thriller,en,After escaping from an Estonian psychiatric fa...,8098.027,Dark Castle Entertainment-Entertainment One-Ea...,2022-07-27,0.0,9572765.0,...,Released,There's always been something wrong with Esther.,6.963,821.0,Isabelle Fuhrman-Julia Stiles-Rossif Sutherlan...,psychopath-family secrets-prequel-murder-imper...,/wSqAXL1EHVJ3MOnJzMhUngc8gFs.jpg,/5GA3vV1aWWHTSDO5eno8V5zDo8r.jpg,606237-818502-744276-721930-1001925-762504-982...,1
1,718930,Bullet Train,Action-Comedy-Thriller,en,Unlucky assassin Ladybug is determined to do h...,7949.491,Columbia Pictures-87North Productions,2022-07-03,90000000.0,231398720.0,...,Released,The end of the line is just the beginning.,7.497,1238.0,Brad Pitt-Joey King-Aaron Taylor-Johnson-Brian...,japan-assassin-based on novel or book-mission-...,/tVxDe01Zy3kZqaZRNiXFGDICdZk.jpg,/y2Ca1neKke2mGPMaHzlCNDVZqsK.jpg,1002564-762504-755566-1002171-682507-884453-55...,2
2,744276,After Ever Happy,Romance-Drama,en,As a shocking truth about a couple's families ...,4017.342,Voltage Pictures-Vertical Entertainment-CalMap...,2022-08-24,0.0,0.0,...,Released,They can't stop loving each other.,6.830,215.0,Josephine Langford-Hero Fiennes Tiffin-Louise ...,based on novel or book-love-teenage crush,/6b7swg6DLqXCO3XUsMnv6RwDMW2.jpg,/rwgmDkIEv8VjAsWx25ottJrFvpO.jpg,20966-676701-419669-804251-766907-619803-67670...,3
3,579974,RRR,Action-Drama,te,A fictional history of two legendary revolutio...,3416.323,Lyca Productions-DVV Entertainment-Pen Studios,2022-03-24,69000000.0,160000000.0,...,Released,"Rise, roar, revolt.",7.774,370.0,N.T. Rama Rao Jr.-Ram Charan-Alia Bhatt-Ajay D...,revolution-liberation-slavery-freedom fighter-...,/wE0I6efAW4cDDmZQWtwZMOW44EJ.jpg,/i0Y0wP8H6SRgjr6QmuwbtQbS24D.jpg,601337-889741-587412-766116-792358-680058-2372...,4
4,532639,Pinocchio,Fantasy-Adventure-Family,en,A wooden puppet embarks on a thrilling adventu...,3239.378,Walt Disney Pictures-Depth of Field,2022-09-07,0.0,0.0,...,Released,The timeless classic.,6.753,800.0,Benjamin Evan Ainsworth-Tom Hanks-Joseph Gordo...,fairy-musical-puppet-live action and animation...,/g8sclIV4gj1TZqUpnL82hKOTK3B.jpg,/nnUQqlVZeEGuCRx8SaoCU4XVHJN.jpg,985939-610150-760741-921360-576925-616037-5799...,5


In [47]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    20000 non-null  int64  
 1   title                 20000 non-null  object 
 2   genres                19644 non-null  object 
 3   original_language     20000 non-null  object 
 4   overview              19651 non-null  object 
 5   popularity            20000 non-null  float64
 6   production_companies  18315 non-null  object 
 7   release_date          19909 non-null  object 
 8   budget                20000 non-null  float64
 9   revenue               20000 non-null  float64
 10  runtime               19990 non-null  float64
 11  status                20000 non-null  object 
 12  tagline               12687 non-null  object 
 13  vote_average          20000 non-null  float64
 14  vote_count            20000 non-null  float64
 15  credits            

In [48]:
movies_df.isnull().sum()

id                         0
title                      0
genres                   356
original_language          0
overview                 349
popularity                 0
production_companies    1685
release_date              91
budget                     0
revenue                    0
runtime                   10
status                     0
tagline                 7313
vote_average               0
vote_count                 0
credits                  526
keywords                3749
poster_path              199
backdrop_path           1280
recommendations         3670
index                      0
dtype: int64

In [49]:
movie_features = movies_df[['index','title', 'genres', 'overview', 'tagline', 'keywords']] #feature selection

for i in movie_features:
    movies_df[i].fillna('', inplace = True)

In [50]:
movie_features.isnull().sum()

index          0
title          0
genres       356
overview     349
tagline     7313
keywords    3749
dtype: int64

In [51]:
feature_strings  =movies_df['title'] + ' ' + movies_df['genres'] + ' ' + movies_df['tagline'] + ' ' + movies_df['keywords'] 

print(feature_strings) 
#we should not perform re with respect o numbers here because iron man, iron man2, iron man3, etc. 
#numbers might be useful to get close cases


0        Orphan: First Kill Horror-Thriller There's alw...
1        Bullet Train Action-Comedy-Thriller The end of...
2        After Ever Happy Romance-Drama They can't stop...
3        RRR Action-Drama Rise, roar, revolt. revolutio...
4        Pinocchio Fantasy-Adventure-Family The timeles...
                               ...                        
19995    Awaken Horror-Thriller-Action A Perfect Vacati...
19996    Lammbock Comedy  hunter-judge-parent child rel...
19997    Scribe Action-Thriller Some things cannot be u...
19998               Quiet Comes the Dawn Horror-Thriller  
19999    The Look of Love Comedy-Drama Based on the out...
Length: 20000, dtype: object


In [52]:
vectorizer = TfidfVectorizer()

feature_vectors = vectorizer.fit_transform(feature_strings)

print(feature_vectors)

  (0, 6998)	0.13426879462141572
  (0, 6513)	0.2736873502389545
  (0, 4984)	0.13173417185173306
  (0, 12207)	0.15849952554808544
  (0, 6477)	0.1927255012136023
  (0, 10379)	0.12031169894177403
  (0, 14857)	0.18906462293902046
  (0, 13846)	0.3895245746964717
  (0, 11911)	0.3199989708022346
  (0, 9382)	0.2491263713390544
  (0, 12582)	0.10269224317990978
  (0, 14626)	0.18850029076682737
  (0, 16509)	0.16241960519379087
  (0, 6851)	0.0790800006119681
  (0, 14865)	0.30674080703925644
  (0, 6512)	0.2491263713390544
  (0, 20714)	0.12222064280436584
  (0, 20847)	0.1646179503449374
  (0, 17472)	0.16701282682662996
  (0, 1934)	0.17077164460489563
  (0, 821)	0.16882640256166137
  (0, 18847)	0.1304872126905347
  (0, 18914)	0.06645810897446462
  (0, 9070)	0.07758664318403449
  (0, 10377)	0.15656522646809495
  :	:
  (19997, 1867)	0.20536654634535279
  (19997, 477)	0.11787152722427059
  (19997, 18914)	0.11554353483243547
  (19998, 4993)	0.5752908839225752
  (19998, 15026)	0.5922841717508274
  (19998, 

In [53]:
similarity = cosine_similarity(feature_vectors)

In [54]:
print(similarity)

[[1.         0.00491388 0.         ... 0.0076788  0.03059936 0.        ]
 [0.00491388 1.         0.07183577 ... 0.0174342  0.03722007 0.07111864]
 [0.         0.07183577 1.         ... 0.         0.         0.06486598]
 ...
 [0.0076788  0.0174342  0.         ... 1.         0.0225142  0.        ]
 [0.03059936 0.03722007 0.         ... 0.0225142  1.         0.02337481]
 [0.         0.07111864 0.06486598 ... 0.         0.02337481 1.        ]]


In [55]:
similarity.shape

(20000, 20000)

In [56]:
movie_name = input('Please enter movie name: ')
movie_name

Please enter movie name: cindrella


'cindrella'

In [57]:
#we will try to get the close match of an input with what we have in db
movie_list = movies_df['title'].to_list()

movie_match = difflib.get_close_matches(movie_name, movie_list)
print(movie_match)

['Cinderella', 'Cinderella', 'Cinderella']


In [58]:
#we only want the one which is the closest
close_match = movie_match[0]
print(close_match)

Cinderella


In [59]:
#get the index of movie which match the input so we can compare the similarity score
close_match_index = movies_df[movies_df['title'] == close_match]['index'].values[0]
print(close_match_index)

371


In [60]:
similarity_score = list(enumerate(similarity[close_match_index]))
similarity_score #similairty score of index 60 with all movies

[(0, 0.008816176283538755),
 (1, 0.03858542984980689),
 (2, 0.0),
 (3, 0.021607619127738214),
 (4, 0.023560403447778728),
 (5, 0.0),
 (6, 0.024587081360624763),
 (7, 0.024597365565854807),
 (8, 0.011950388349639018),
 (9, 0.01212808105075428),
 (10, 0.022098451015938332),
 (11, 0.010359779157433588),
 (12, 0.11514572890288638),
 (13, 0.0),
 (14, 0.01806159084700916),
 (15, 0.01239929164800618),
 (16, 0.009897195404669662),
 (17, 0.014862834768811319),
 (18, 0.03507625018087497),
 (19, 0.012416767787673135),
 (20, 0.03261263555553059),
 (21, 0.0),
 (22, 0.03130292743012561),
 (23, 0.03529576920817068),
 (24, 0.03231792969299959),
 (25, 0.21802217408646896),
 (26, 0.012655541509653461),
 (27, 0.06403163807685308),
 (28, 0.045983823183425136),
 (29, 0.007652201648706769),
 (30, 0.006789949699147363),
 (31, 0.1016478198270184),
 (32, 0.010150384529121604),
 (33, 0.03575086388491101),
 (34, 0.015979289691727478),
 (35, 0.0),
 (36, 0.02527852102718094),
 (37, 0.0),
 (38, 0.01600597468107977)

In [61]:
sorted_similairity = sorted(similarity_score, reverse= True , key = lambda x: x[1])
print(sorted_similairity) #sorted by similairity score

[(371, 1.0), (458, 1.0), (8629, 0.28299632450056417), (13089, 0.2544634949834182), (19230, 0.2209725012713574), (25, 0.21802217408646896), (12756, 0.19984284150920176), (7135, 0.1983584758735294), (14881, 0.19565819203486606), (17506, 0.1901959089602385), (19090, 0.1901959089602385), (8215, 0.18907748144222386), (19633, 0.18656845756088772), (6901, 0.18479692542846116), (9143, 0.17986590962878196), (6903, 0.17956804467807838), (17160, 0.17805441011323192), (14200, 0.17619367432606373), (6766, 0.1755070547133022), (2150, 0.17519127291099743), (108, 0.1746072826648847), (9398, 0.17377220465001791), (12893, 0.17368049272837371), (3719, 0.17314387814689014), (4770, 0.17244711364157295), (9244, 0.1719028172880343), (3116, 0.1691239709492023), (11183, 0.16873015880483117), (392, 0.16660215918204646), (5955, 0.16190123655182004), (11140, 0.16046866306486537), (15076, 0.1590865115744726), (4976, 0.15903107736818473), (7579, 0.15888940079671685), (12217, 0.15639891144229906), (3665, 0.155832223

In [63]:
print('Movies suggested for you: \n')

i = 1 #iterations

for movie in sorted_similairity :
    index = movie[0]
    title_of_movie = movies_df[movies_df.index == index]['title'].values[0]
    if(i <= 30):
        print(i,'.', title_of_movie)
        i = i+1

Movies suggested for you: 

1 . Last Shoot Out
2 . Last Shoot Out
3 . Shoot to Live
4 . 44 Minutes: The North Hollywood Shoot-Out
5 . The Last Will Be the Last
6 . Wire Room
7 . Man of the East
8 . The Last Stand
9 . The Whole Ten Yards
10 . Every Last Secret
11 . Every Last Secret
12 . Quigley Down Under
13 . Dr. Giggles
14 . City Hunter
15 . Manhunt
16 . Serenity
17 . Shoot Out
18 . In & Out
19 . Soy Luna: The Last Concert
20 . Last Survivors
21 . Memory
22 . The Last Days of American Crime
23 . Guns of the Magnificent Seven
24 . Last Summer
25 . No Tears for the Dead
26 . The Last Warrior
27 . God's Country
28 . Every Last One of Them
29 . Desperate Riders
30 . The Squad


In [64]:
movies_df[movies_df.index == 458]['title']

458    Last Shoot Out
Name: title, dtype: object